# Pre-process non-labeled dataset

## Convert to nifti

In [ ]:
import os, glob
import subprocess
import numpy as np

input_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset/'
output_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop
# i=0
for folder1 in sorted(os.listdir(input_dir)):
    # i+=1
    filename = folder1
    for folder2 in os.listdir(input_dir+folder1):
        # if i==10: break
        if 't1' in folder2 and not folder2.startswith('.'):
            input_dicom_folder = input_dir+folder1+'/'+folder2
            
            # output
            output_nifti_folder = output_dir+folder1
            if not os.path.exists(output_nifti_folder):
                os.makedirs(output_nifti_folder)

            # dcm2niix
            # dicom2nifti.dicom_series_to_nifti(input_dicom_folder, output_nifti_folder+filename, reorient_nifti=True)
            cmd = ["dcm2niix", "-f", filename, "-z", "y", "-o", output_nifti_folder, input_dicom_folder]
            process = subprocess.Popen(cmd, stdout=subprocess.PIPE)  # pass the list as input to Popen
            _ = process.communicate()[0]  # the [0] is to return just the output, because otherwise it would be outs, errs = proc.communicate()
            
            # Dealing with files in that folder
            # for f in glob.glob(input_dir+folder1+'/anat/'+folder1+'_T1.nii.gz'):
            #     os.remove(f)

## Crop eye center

In [ ]:
import numpy
import os,sys
import nibabel as nib
import SimpleITK as sitk
import pandas as pd
import numpy as np

def cropVolumeImage(image,x,y,z,border1,border2,border3):
    
    xSize, ySize, zSize = image.GetSize()
    #xMin,yMin,zMin = image.TransformPhysicalPointToIndex([x-border,y-border,z-border])
    #xMax,yMax,zMax = image.TransformPhysicalPointToIndex([x+border,y+border,z+border])

    #x,y,z = image.TransformPhysicalPointToIndex([x,y,z])
    xMin = x - int(border1/2) 
    xMax = x + int(border1/2) 

    yMin = y - int(border2/2) 
    yMax = y + int(border2/2)  

    zMin = z - int(border3/2) 
    zMax = z + int(border3/2)
    
    # Define limits
    if xMin< 0 :
        xMin = 0  
    if xMax> xSize:
        xMax = xSize

    if yMin< 0 :
        yMin = 0
    if yMax > ySize:
        yMax = ySize

    if zMin< 0 :
        zMin = 0
    if zMax > zSize:
        zMax = zSize

    imageReturn = image[int(xMin):int(xMax),int(yMin):int(yMax),int(zMin):int(zMax)]
    return imageReturn  

# Paths
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_path = main_path + 'non_labeled_dataset_nifti/'
output_path = main_path + 'non_labeled_dataset_nifti_cropped/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Variables to crop 
eye_center = np.array([112, 173, 67]) # Mean coordinates of the center of the right eye (base of the cornea to catch optic nerve too)
croping_cube = np.array([96, 96, 96])

i = 0
for folder1 in sorted(os.listdir(input_path)):
    filename = folder1
    # Path to the folder with images
    input_image_path = input_path + folder1 + '/' + folder1 + '.nii.gz'
    output_image_path = output_path + folder1 + '.nii.gz'
    print(input_image_path)

    input_image = sitk.ReadImage(input_image_path)

    cropped_image = cropVolumeImage(input_image, eye_center[0], eye_center[1], eye_center[2], croping_cube[0], croping_cube[1], croping_cube[2])

    sitk.WriteImage(cropped_image, output_image_path)

    i+=1
    if (i==10):
        break

## Registration 1 - Full images

In [ ]:
import os, glob

# Paths - full image (1st iteration)
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_path = main_path + 'non_labeled_dataset_nifti/'
output_path = main_path + 'non_labeled_dataset_nifti_reg/'
template = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0.nii.gz' # full image template - fixed image (reference)
template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/sub-29_labels2template5_2.nii.gz'

# i = 0
for folder1 in sorted(os.listdir(input_path)):
    if folder1 not in sorted(os.listdir(output_path)):
    
        # Full images paths
        input_image_path = input_path + folder1 + '/' + folder1 + '.nii.gz' # moving image - full
        output_image_path = output_path + folder1 + '/'

        if not os.path.exists(output_image_path):
            os.makedirs(output_image_path)

        ## antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = 'antsRegistrationSyNQuick.sh -d 3' + \
        ' -m ' + input_image_path                     + \
        ' -f ' + template                             + \
        ' -t ' + 's'                                  + \
        ' -o ' + output_image_path                    + \
        ' -n ' + '16'
        # print(command1)
        # os.system(command1)

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = 'antsApplyTransforms -d 3 '                       + \
        ' -i ' +  template_labels                                    + \
        ' -r ' +  input_image_path                                   + \
        ' -t ' + '[' + output_image_path + '0GenericAffine.mat, 1 ]' + \
        ' -t ' + output_image_path + '1InverseWarp.nii.gz'           + \
        ' -n ' + 'MultiLabel'                                        + \
        ' -o ' +  output_image_path + 'labels.nii.gz'                + \
        ' --float 0 --verbose 1'
        # print(command2)
        # os.system(command2)

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

        # i+=1
        # if (i==1):
        #     break

## Crop

In [ ]:
import numpy as np
import os,sys
import nibabel as nib
import SimpleITK as sitk
import pandas as pd

# Paths
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_image_path = main_path + 'non_labeled_dataset_nifti/'
input_label_path = main_path + 'non_labeled_dataset_nifti_reg/'
output_path = main_path + 'non_labeled_dataset_nifti_cropped/'

# i=0
for folder1 in sorted(os.listdir(input_label_path)):
    if folder1+'_cropped.nii.gz' not in list(sorted(os.listdir(output_path))):

        image_path = input_image_path + folder1 + '/' + folder1 + '.nii.gz' # image
        labels_path = input_label_path + folder1 + '/labels.nii.gz' # labels
        bound = 15 # boundary for the bounding box (margins)

        image = sitk.ReadImage(image_path)
        all_segments = sitk.ReadImage(labels_path)
        image_x_size, image_y_size, image_z_size = image.GetSize()
        print(f"image_x_size {image_x_size} image_y_size {image_y_size} image_z_size {image_z_size}")

        # Mask
        all_segments_mask = all_segments > 0
        # sitk.WriteImage(all_segments_mask, base_dir+folder+'/input/'+folder+'_labels_mask.nii.gz')

        # Bounding box
        lsif = sitk.LabelStatisticsImageFilter() # It requires intensity and label images
        lsif.Execute(image, all_segments_mask) # Mask! Where all the labels are 1!
        bounding_box = np.array(lsif.GetBoundingBox(1)) # GetBoundingBox(label)
        print(f"Bounding box:  {bounding_box}") # [xmin, xmax, ymin, ymax, zmin, zmax]
        bounding_box_expanded = bounding_box.copy()
        bounding_box_expanded[0::2] -= bound # even indexes
        bounding_box_expanded[1::2] += bound # odd indexes
        print(f"Expanded bounding box: {bounding_box_expanded}")

        # Limits
        if bounding_box_expanded[0] < 0: bounding_box_expanded[0] = 0
        if bounding_box_expanded[1] > image_x_size: bounding_box_expanded[1] = image_x_size
        if bounding_box_expanded[2] < 0: bounding_box_expanded[2] = 0
        if bounding_box_expanded[3] > image_y_size: bounding_box_expanded[3] = image_y_size
        if bounding_box_expanded[4] < 0: bounding_box_expanded[4] = 0
        if bounding_box_expanded[5] > image_z_size: bounding_box_expanded[5] = image_z_size
        print(f"Expanded bounding box after limits: {bounding_box_expanded}")

        # Crop
        image_crop = image[int(bounding_box_expanded[0]):int(bounding_box_expanded[1]), # x
                            int(bounding_box_expanded[2]):int(bounding_box_expanded[3]), # y
                            int(bounding_box_expanded[4]):int(bounding_box_expanded[5])] # z
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        sitk.WriteImage(image_crop, output_path + folder1 + '_cropped.nii.gz')

        # i+=1
        # if (i==1):
        #     break             

## Registration 2 - Cropped images

In [ ]:
import os

# Paths - cropped images (2nd iteration)
input_path = '/home/jaimebarranco/Downloads/MREye_Studybrain_1_segmentations/atlas/cropped/'
output_path = '/home/jaimebarranco/Downloads/MREye_Studybrain_1_segmentations/atlas/reg2_SyN/'
template = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0_cropped_15vox.nii.gz' # cropped image template
template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/Probability_Maps/prob_map_cropped_th0_2.nii.gz'

for folder in sorted(os.listdir(input_path)):
        # Cropped images paths
        input_image_path = input_path + folder + '/' + folder + '_cropped.nii.gz' # moving image - cropped
        output_image_path = output_path + folder + '/'

        if not os.path.exists(output_image_path):
            os.makedirs(output_image_path)

        ## antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = 'antsRegistrationSyNQuick.sh -d 3' + \
        ' -m ' + input_image_path                     + \
        ' -f ' + template                             + \
        ' -t ' + 's'                                  + \
        ' -o ' + output_image_path                    + \
        ' -n ' + '16'
        # print(command1)
        os.system(command1)

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = 'antsApplyTransforms -d 3 '                       + \
        ' -i ' +  template_labels                                    + \
        ' -r ' +  input_image_path                                   + \
        ' -t ' + '[' + output_image_path + '0GenericAffine.mat, 1 ]' + \
        ' -t ' + output_image_path + '1InverseWarp.nii.gz'           + \
        ' -n ' + 'MultiLabel'                                        + \
        ' -o ' +  output_image_path + 'labels.nii.gz'                + \
        ' --float 0 --verbose 1'
        # print(command2)
        os.system(command2)

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

# nnUnet results

Registration of atlas and nnUNet labels to crop and allign them to the same space

In [ ]:
import os

# Paths
reg_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg_2/'
nnunet_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing/'
output_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing_cropped_reg/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

i = 0
for folder1 in sorted(os.listdir(reg_dir)):
    # if folder1 not in sorted(os.listdir(output_path)):
    if folder1 == '2022160101206':
        # Images' paths
        fixed_image_path = f'{reg_dir}{folder1}/labels.nii.gz' # fixed image (atlas)
        moving_image_path = f'{nnunet_dir}AEye_{folder1}.nii.gz' # moving image (nnunet)
        output_image_path = f'{output_path}{folder1}/'
        if not os.path.exists(output_image_path):
            os.makedirs(output_image_path)

        ## antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = 'antsRegistrationSyNQuick.sh -d 3' + \
        ' -m ' + moving_image_path                     + \
        ' -f ' + fixed_image_path                             + \
        ' -t ' + 's'                                  + \
        ' -o ' + output_image_path                    + \
        ' -n ' + '16'
        # print(command1)
        os.system(command1)

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = 'antsApplyTransforms -d 3 '                       + \
        ' -i ' +  moving_image_path                                    + \
        ' -r ' +  fixed_image_path                                   + \
        ' -t ' + output_image_path + '1Warp.nii.gz'           + \
        ' -t ' + output_image_path + '0GenericAffine.mat' + \
        ' -o ' +  output_image_path + 'cropped_alligned.nii.gz'                + \
        ' --float 0 --verbose 0' +\
        ' -n ' + 'Multilabel'                                        
        # print(command2)
        os.system(command2)

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

        i+=1
        # if (i==1):
        #     break

# New data

## Convert to nifti

In [ ]:
import os
import subprocess
import glob

input_dir = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_Registration_Barranco'
output_dir = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

dcm_files = glob.glob(os.path.join(input_dir, '**', '*.dcm'), recursive=True)
dcm_folders = sorted(set(os.path.dirname(dcm_file) for dcm_file in dcm_files))


dcm2niix

In [ ]:
# Loop
for folder in sorted(dcm_folders):

    sex = folder.split('/')[-3]  # male or female (to name the folders)
    id = folder.split('/')[-2]  # id (to name the folders)
    seq = folder.split('/')[-1]  # sequence (to name the folders)
    img_name = id + '_' + seq

    output_path = os.path.join(output_dir, 'male' if sex == 'Male' else 'female')
    os.makedirs(output_path, exist_ok=True)

    # dcm2niix
    # dicom2nifti.dicom_series_to_nifti(input_dicom_folder, output_nifti_folder+filename, reorient_nifti=True)
    cmd = ["dcm2niix", "-f", img_name, "-z", "y", "-o", output_path, "-g", "n", "-m", "y", "-s", "y", folder]  # it's generating more files
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE)  # pass the list as input to Popen
    _ = process.communicate()[0]  # the [0] is to return just the output, because otherwise it would be outs, errs = proc.communicate()


remove extra files from dcm2niix

In [ ]:
import os
import re

# Pattern to identify files ending with a letter before .nii.gz or .json
pattern = re.compile(r".*[a-z]\.(nii\.gz|json)$")

# Walk through all directories and subdirectories
for root, dirs, files in os.walk(output_dir):
    for filename in files:
        if pattern.match(filename):
            file_path = os.path.join(root, filename)
            os.remove(file_path)
            # print(f"Removed {file_path}")


remove all the files in output_dir

In [ ]:
# Dealing with files in that folder
for file in glob.glob(os.path.join(output_dir, '**', '*.nii.gz'), recursive=True):
    os.remove(file)
for file in glob.glob(os.path.join(output_dir, '**', '*.json'), recursive=True):
    os.remove(file)

## Registration 1 - Full images

binaries

In [ ]:
import os

os.environ['PATH'] += ':/opt/ANTs/bin' # ensure the ANTs binaries are in $PATH
os.environ['ANTSPATH'] = '/opt/ANTs/bin/'
# os.system("echo $PATH")
# os.system("which antsApplyTransforms")
# os.system("antsApplyTransforms -h")
print(os.environ['PATH'])

Registration with ANTs

In [ ]:
import os, subprocess, glob

# Paths - full image (1st iteration)
input_path = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_flipped'
output_path = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/reg1_flipped'
template = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0.nii.gz'  # full image template - fixed image (reference)
template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/Probability_Maps/prob_map_th0.nii.gz'  # mix of labels from the 5 subjects
# template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/sub-29_labels2template5_2.nii.gz'

imgs_paths = glob.glob(os.path.join(input_path, '**', '*.nii.gz'), recursive=True)

for img in sorted(imgs_paths):

    img_name = os.path.basename(img).split('.')[0]
    output_reg_path = os.path.join(output_path, img_name) + '/'
    os.makedirs(output_reg_path, exist_ok=True)

    # antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
    command1 = ['antsRegistrationSyNQuick.sh', 
                '-d', '3',
                '-m', img,
                '-f', template,
                '-t', 's',
                '-o', output_reg_path,
                '-n', '16',
                '-j', '1']
    # print(command1)
    # os.system(command1)
    process = subprocess.Popen(command1, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(f'Error: {stderr.decode()}')
    else:
        print(stdout.decode())

    # antsApplyTransforms with inverse transform to get the template labels into subject space
    command2 = ['antsApplyTransforms', '-d', '3',
                '-i', template_labels,
                '-r', img,
                '-t', '[' + output_reg_path + '0GenericAffine.mat, 1 ]',
                '-t', output_reg_path + '1InverseWarp.nii.gz',
                '-n', 'MultiLabel',
                '-o', output_reg_path + img_name + '_labels.nii.gz',
                '--float', '0',
                '--verbose', '1']
    # print(command2)
    # os.system(command2)
    process = subprocess.Popen(command2, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(f'Error: {stderr.decode()}')
    else:
        print(stdout.decode())

    # Dealing with files in that folder
    # for f in glob.glob(output_image_path + 'labels.nii.gz'):
    #     os.remove(f)

Allignment image - labels

In [ ]:
import os
import glob
import nibabel as nb

# Paths
input_path = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_flipped'
output_path = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/reg1_flipped'
imgs_paths = glob.glob(os.path.join(input_path, '**', '*.nii.gz'), recursive=True)
imgs_paths = [img for img in imgs_paths if img != '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_flipped/male/SD_T1_SE_FS_flipped.nii.gz']  # problematic image (no registration)
imgs_paths = sorted(imgs_paths)
labels_paths = glob.glob(os.path.join(output_path, '**', '*_labels.nii.gz'), recursive=True)
labels_paths = sorted(labels_paths)

for i in sorted(range(len(imgs_paths))):  # or labels_paths

    img = imgs_paths[i]
    labels = labels_paths[i]
    
    filename = os.path.basename(img).split('.')[0]

    # Load reference image
    image = nb.load(img)  # load image

    # Load t1 image (not alligned)
    t1 = nb.load(labels)  # load labels
    print(t1.affine)

    # Copy affine from template to t1
    nii = nb.Nifti1Image(t1.dataobj, image.affine, image.header)
    print(t1.affine)

    # Output
    nii.to_filename(os.path.dirname(labels) + '/' + filename + '_labels_aff.nii.gz')
    
    # Dealing with files in that folder
    # for f in glob.glob(os.path.dirname(labels) + '/' + os.path.basename(labels).split('.')[0] + '*_labels_aff.nii.gz'):
    #     os.remove(f)

Crop

In [ ]:
import numpy as np
import os
import glob
import SimpleITK as sitk

# Paths
input_image_path = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_flipped'
input_label_path = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/reg1_flipped'
output_path = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_cropped_flipped'

imgs_paths = glob.glob(os.path.join(input_image_path, '**', '*.nii.gz'), recursive=True)
labels_paths = glob.glob(os.path.join(input_label_path, '**', '*_labels_aff.nii.gz'), recursive=True)

# remove problematic cases
imgs_paths = [img for img in imgs_paths if img != '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_flipped/male/SD_T1_SE_FS_flipped.nii.gz']  # problematic image (no registration)
imgs_paths = [img for img in imgs_paths if img != '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_flipped/female/SA_T1_LAVA_STAR_flipped.nii.gz']  # problematic image (no labels transposed)
labels_paths = [lab for lab in labels_paths if lab !='/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/reg1_flipped/female/SA_T1_LAVA_STAR_flipped/SA_T1_LAVA_STAR_flipped_labels_aff.nii.gz']
imgs_paths = [img for img in imgs_paths if img != '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_flipped/male/RH_wT1_FSE_flipped.nii.gz']  # problematic image (no labels transposed)
labels_paths = [lab for lab in labels_paths if lab !='/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/reg1_flipped/male/RH_wT1_FSE_flipped/RH_wT1_FSE_flipped_labels_aff.nii.gz']

imgs_paths = sorted(imgs_paths)
labels_paths = sorted(labels_paths)

image_names = [os.path.basename(img).split('.')[0] for img in imgs_paths]

# Define the integers yourself
bounds = [100, 50, 200, 75, 200, 50, 50, 20, 60, 200, 50, 50]  # specific bounds for each image
bounds_dict = {name: bounds[i] for i, name in enumerate(image_names)}

for i in sorted(range(len(imgs_paths))):  # or labels_paths

    img = imgs_paths[i]
    labels = labels_paths[i]

    sex = img.split('/')[-2]
    filename_img = os.path.basename(img).split('.')[0]
    filename_labels = os.path.basename(labels).split('.')[0]
    print(f'sex: {sex} || image: {filename_img} || labels: {filename_labels}')

    image = sitk.ReadImage(img)
    all_segments = sitk.ReadImage(labels)
    image_x_size, image_y_size, image_z_size = image.GetSize()
    print(f"image_x_size {image_x_size} image_y_size {image_y_size} image_z_size {image_z_size}")

    bound = bounds_dict[filename_img]
    print(f"Bound: {bound}")

    # Mask
    all_segments_mask = all_segments > 0
    # sitk.WriteImage(all_segments_mask, base_dir+folder+'/input/'+folder+'_labels_mask.nii.gz')

    # Bounding box
    lsif = sitk.LabelStatisticsImageFilter() # It requires intensity and label images
    lsif.Execute(image, all_segments_mask) # Mask! Where all the labels are 1!
    bounding_box = np.array(lsif.GetBoundingBox(1)) # GetBoundingBox(label)
    print(f"Bounding box:  {bounding_box}") # [xmin, xmax, ymin, ymax, zmin, zmax]
    bounding_box_expanded = bounding_box.copy()
    bounding_box_expanded[0::2] -= bound # even indexes
    bounding_box_expanded[1::2] += bound # odd indexes
    print(f"Expanded bounding box: {bounding_box_expanded}")

    # Limits
    if bounding_box_expanded[0] < 0: bounding_box_expanded[0] = 0
    if bounding_box_expanded[1] > image_x_size: bounding_box_expanded[1] = image_x_size
    if bounding_box_expanded[2] < 0: bounding_box_expanded[2] = 0
    if bounding_box_expanded[3] > image_y_size: bounding_box_expanded[3] = image_y_size
    if bounding_box_expanded[4] < 0: bounding_box_expanded[4] = 0
    if bounding_box_expanded[5] > image_z_size: bounding_box_expanded[5] = image_z_size
    print(f"Expanded bounding box after limits: {bounding_box_expanded} \n")

    # Crop
    image_crop = image[int(bounding_box_expanded[0]):int(bounding_box_expanded[1]), # x
                        int(bounding_box_expanded[2]):int(bounding_box_expanded[3]), # y
                        int(bounding_box_expanded[4]):int(bounding_box_expanded[5])] # z
    os.makedirs(output_path + '/' + sex, exist_ok=True)
    sitk.WriteImage(image_crop, output_path + '/' + sex + '/' + filename_img + '_cropped.nii.gz')            

## Registration 2 - Eye template for male and females separately

In [ ]:
import os
import subprocess
import glob

sex = ['female', 'male']

for s in sex:

    # Paths - cropped images (2nd iteration)
    input_path = f'/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_cropped/{s}'
    output_path = f'/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/reg2_pro/{s}/'
    template = f'/mnt/sda1/Repos/a-eye/Output/eye_model/{s}/output/templateConstruction/template0.nii.gz'  # cropped image template
    template_labels = f'/mnt/sda1/Repos/a-eye/Output/eye_model/{s}/output/maps/nnunet/max_prob_map.nii.gz'

    imgs_paths = glob.glob(os.path.join(input_path, '**', '*.nii.gz'), recursive=True)

    for img in sorted(imgs_paths):

        img_name = os.path.basename(img).split('.')[0]
        output_reg_path = os.path.join(output_path, img_name) + '/'
        os.makedirs(output_reg_path, exist_ok=True)

        # antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = ['antsRegistrationSyNQuick.sh', 
                    '-d', '3',
                    '-m', img,
                    '-f', template,
                    '-t', 'b',
                    '-s', '10',
                    '-r', '64',
                    '-g', '0.15',
                    '-o', output_reg_path,
                    '-n', '16',
                    '-j', '1']
        print(command1)
        # os.system(command1)
        process = subprocess.Popen(command1, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        if process.returncode != 0:
            print(f'Error: {stderr.decode()}')
        else:
            print(stdout.decode())

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = ['antsApplyTransforms', '-d', '3',
                    '-i', template_labels,
                    '-r', img,
                    '-t', '[' + output_reg_path + '0GenericAffine.mat, 1 ]',
                    '-t', output_reg_path + '1InverseWarp.nii.gz',
                    '-n', 'MultiLabel',
                    '-o', output_reg_path + img_name + '_labels.nii.gz',
                    '--float', '0',
                    '--verbose', '1']
        print(command2)
        # os.system(command2)
        process = subprocess.Popen(command2, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        if process.returncode != 0:
            print(f'Error: {stderr.decode()}')
        else:
            print(stdout.decode())

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

# EO Analysis

## Volumetry

reference

In [ ]:
import os
import glob
import pandas as pd

# atlases directories
female_atlas_img_dir = f'/mnt/sda1/Repos/a-eye/Output/eye_model/female/output/registrationToTemplate'
female_atlas_labels_dir = f'/mnt/sda1/Repos/a-eye/Output/eye_model/female/output/maps/nnunet'
male_atlas_img_dir = f'/mnt/sda1/Repos/a-eye/Output/eye_model/male/output/registrationToTemplate'
male_atlas_labels_dir = f'/mnt/sda1/Repos/a-eye/Output/eye_model/male/output/maps/nnunet'

# volumes CSVs (ref)
female_atlas_volumes = pd.read_csv(f'{female_atlas_labels_dir}/volumes.csv')
male_atlas_volumes = pd.read_csv(f'{male_atlas_labels_dir}/volumes.csv')

# Mapping of integer labels to descriptive labels
label_mapping = {
    0.0: 'bg',
    1.0: 'lens',
    2.0: 'globe',
    3.0: 'opt_ner',
    4.0: 'int_fat',
    5.0: 'ext_fat',
    6.0: 'lat_rm',
    7.0: 'med_rm',
    8.0: 'inf_rm',
    9.0: 'sup_rm'
}

# Apply the mapping to the 'Label' column
female_atlas_volumes['label'] = female_atlas_volumes['label'].map(label_mapping)
male_atlas_volumes['label'] = male_atlas_volumes['label'].map(label_mapping)

right eyes

In [ ]:
import nibabel as nib
import numpy as np

sex = ['female', 'male']

results_df = pd.DataFrame(columns=['Subject', 'Label', 'VolumeSubject', 'VolumeReference', 'VolumeDifference', 'VoxelDims', 'Sex'])

for s in sex:

    # reference volume per structure per sex
    ref_volumes = female_atlas_volumes if s == 'female' else male_atlas_volumes

    # Paths
    labels_dir = f'/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/reg2/{s}/'
    labels_paths = glob.glob(os.path.join(labels_dir, '**', '*_labels.nii.gz'), recursive=True)

    for l in sorted(labels_paths):

        labels_name = os.path.basename(l).split('.')[0].replace('_cropped_labels', '')

        # Read image with nibabel
        labels = nib.load(l)
        labels_data = labels.get_fdata()

        # Get voxel dimensions
        voxel_dims = labels.header.get_zooms()

        # count voxels per label
        unique, counts = np.unique(labels_data, return_counts=True)

        # Get the volume of each label
        volumes = counts * np.prod(voxel_dims)  # in mm³
        
        # Create a dictionary with the volumes
        label_volumes = dict(zip(unique, volumes))

        # Compute volume difference per structure
        vd = []
        volume_refs = []
        for label in unique:
            ref_volume = ref_volumes.loc[ref_volumes['label'] == label_mapping[label], 'count'].values[0]
            volume_refs.append(ref_volume)
            volume_diff = 2 * (label_volumes[label] - ref_volume) / (label_volumes[label] + ref_volume)
            # if it's positive, it means that the volume of the subject is bigger than the reference
            vd.append(volume_diff)

        # Create a DataFrame from the dictionary
        temp_df = pd.DataFrame(list(label_volumes.items()), columns=['Label', 'VolumeSubject'])
        temp_df['Subject'] = labels_name
        temp_df['VolumeReference'] = volume_refs
        temp_df['VolumeDifference'] = vd        
        temp_df['VoxelDims'] = str(voxel_dims)
        temp_df['Sex'] = s

        # Concatenate the temporary DataFrame with the results_df DataFrame
        results_df = pd.concat([results_df, temp_df], ignore_index=True)

# Apply the mapping to the 'Label' column
results_df['Label'] = results_df['Label'].map(label_mapping)

# Save the DataFrame to a CSV file
# results_df.to_csv('/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/results_volumetry_right_eyes.csv', index=False)

In [ ]:
import pandas as pd

df_right_eye = pd.read_csv('/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/results_volumetry_right_eyes.csv')

plot

left eyes

In [ ]:
import nibabel as nib
import numpy as np

sex = ['female', 'male']

results_df = pd.DataFrame(columns=['Subject', 'Label', 'VolumeSubject', 'VolumeReference', 'VolumeDifference', 'VoxelDims', 'Sex'])

for s in sex:

    # reference volume per structure per sex
    ref_volumes = female_atlas_volumes if s == 'female' else male_atlas_volumes

    # Paths
    labels_dir = f'/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/reg2_flipped/{s}/'
    labels_paths = glob.glob(os.path.join(labels_dir, '**', '*_labels.nii.gz'), recursive=True)

    for l in sorted(labels_paths):

        labels_name = os.path.basename(l).split('.')[0].replace('_cropped_labels', '')

        # Read image with nibabel
        labels = nib.load(l)
        labels_data = labels.get_fdata()

        # Get voxel dimensions
        voxel_dims = labels.header.get_zooms()

        # count voxels per label
        unique, counts = np.unique(labels_data, return_counts=True)

        # Get the volume of each label
        volumes = counts * np.prod(voxel_dims)  # in mm³
        
        # Create a dictionary with the volumes
        label_volumes = dict(zip(unique, volumes))

        # Compute volume difference per structure
        vd = []
        volume_refs = []
        for label in unique:
            ref_volume = ref_volumes.loc[ref_volumes['label'] == label_mapping[label], 'count'].values[0]
            volume_refs.append(ref_volume)
            volume_diff = 2 * (label_volumes[label] - ref_volume) / (label_volumes[label] + ref_volume)
            # if it's positive, it means that the volume of the subject is bigger than the reference
            vd.append(volume_diff)

        # Create a DataFrame from the dictionary
        temp_df = pd.DataFrame(list(label_volumes.items()), columns=['Label', 'VolumeSubject'])
        temp_df['Subject'] = labels_name
        temp_df['VolumeReference'] = volume_refs
        temp_df['VolumeDifference'] = vd        
        temp_df['VoxelDims'] = str(voxel_dims)
        temp_df['Sex'] = s

        # Concatenate the temporary DataFrame with the results_df DataFrame
        results_df = pd.concat([results_df, temp_df], ignore_index=True)

# Apply the mapping to the 'Label' column
results_df['Label'] = results_df['Label'].map(label_mapping)

# Save the DataFrame to a CSV file
# results_df.to_csv('/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/results_volumetry_left_eyes.csv', index=False)

In [ ]:
import pandas as pd

df_left_eye = pd.read_csv('/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/results_volumetry_left_eyes.csv')

## Surface

### Extract meaningful 2D images

Reference

In [ ]:
# female
c3d /mnt/sda1/Repos/a-eye/Output/eye_model/female/output/templateConstruction/template0.nii.gz -slice z 59 -o /mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/refs_2D/female_img.nii.gz
c3d /mnt/sda1/Repos/a-eye/Output/eye_model/female/output/maps/nnunet/max_prob_map.nii.gz -slice z 59 -o /mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/refs_2D/female_seg.nii.gz
# male
c3d /mnt/sda1/Repos/a-eye/Output/eye_model/male/output/templateConstruction/template0.nii.gz -slice z 36 -o /mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/refs_2D/male_img.nii.gz
c3d /mnt/sda1/Repos/a-eye/Output/eye_model/male/output/maps/nnunet/max_prob_map.nii.gz -slice z 36 -o /mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/refs_2D/male_seg.nii.gz


Subjects

In [ ]:
import os
import glob

input_dir = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_cropped'
imgs_paths = glob.glob(os.path.join(input_dir, '**', '*.nii.gz'), recursive=True)
imgs_paths = sorted(imgs_paths)
image_names = [os.path.basename(img).split('.')[0] for img in imgs_paths]

output_dir = '/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_cropped_2D'
os.makedirs(output_dir, exist_ok=True)

z_slices = [9, 33, 0, 0, 0, 27, 32, 2, 3, 8, 6, 2]  # specific bounds for each image
z_slices_dict = {name: z_slices[i] for i, name in enumerate(image_names)}

for i in sorted(imgs_paths):

        sex = i.split('/')[-2]
        filename_img = os.path.basename(i).split('.')[0]
        print(f'sex: {sex} || image: {filename_img}')

        os.makedirs(output_dir + '/' + sex, exist_ok=True)
        output_image_path = output_dir + '/' + sex + '/' + filename_img + '_2D.nii.gz'

        z_slice = z_slices_dict[filename_img]
        print(f"z-slice: {z_slice}")

        command = f'c3d {i} -slice z {z_slice} -o {output_image_path}'
        print(command)
        # os.system(command)


Registration 2D

In [ ]:
import os
import subprocess
import glob

sex = ['female', 'male']

for s in sex:

    # Paths - cropped images (2nd iteration)
    input_path = f'/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/imgs_cropped_2D/{s}'
    output_path = f'/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/reg2_2D_pro/{s}/'
    template = f'/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/refs_2D/{s}_img.nii.gz'  # cropped image template
    template_labels = f'/mnt/sda1/Repos/a-eye/Data/adrian_luyken/EO_study/refs_2D/{s}_seg.nii.gz'

    imgs_paths = glob.glob(os.path.join(input_path, '**', '*.nii.gz'), recursive=True)

    for img in sorted(imgs_paths):

        img_name = os.path.basename(img).split('.')[0]
        output_reg_path = os.path.join(output_path, img_name) + '/'
        os.makedirs(output_reg_path, exist_ok=True)

        # antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = ['antsRegistrationSyNQuick.sh', 
                    '-d', '2',
                    '-m', img,
                    '-f', template,
                    '-t', 'b',
                    '-s', '10',
                    '-r', '64',
                    '-g', '0.15',
                    '-o', output_reg_path,
                    '-n', '16',
                    '-j', '1']
        print(command1)
        # os.system(command1)
        process = subprocess.Popen(command1, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        if process.returncode != 0:
            print(f'Error: {stderr.decode()}')
        else:
            print(stdout.decode())

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = ['antsApplyTransforms', '-d', '2',
                    '-i', template_labels,
                    '-r', img,
                    '-t', '[' + output_reg_path + '0GenericAffine.mat, 1 ]',
                    '-t', output_reg_path + '1InverseWarp.nii.gz',
                    '-n', 'MultiLabel',
                    '-o', output_reg_path + img_name + '_labels.nii.gz',
                    '--float', '0',
                    '--verbose', '1']
        print(command2)
        # os.system(command2)
        process = subprocess.Popen(command2, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        if process.returncode != 0:
            print(f'Error: {stderr.decode()}')
        else:
            print(stdout.decode())

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

# Registration to MNI space

antsEyeExtraction.sh

In [ ]:
antsEyeExtraction.sh -d 3 -a ../input/T1.nii.gz -e $( python -c "from templateflow import api; print(str(api.get('MNI152NLin2009cAsym', suffix='T1w', desc=None, resolution=1)))" ) -f $( python -c "from templateflow import api; print(str(api.get('MNI152NLin2009cAsym', suffix='mask', desc='eye', resolution=1)))" ) -g '/mnt/sda1/ANTs/input/all_segments.nii.gz'  -o output -k 1

antsEyeExtraction.sh -d 3 -a ../input/T1.nii.gz -e ../input/colin27/tpl-MNIColin27_T1w.nii.gz -f ../input/mni152/tpl-MNI152NLin2009cAsym_res-01_desc-eye_mask.nii.gz -g '/mnt/sda1/ANTs/input/all_segments.nii.gz'  -o output -k 1

antsEyeExtraction.sh -d 3 -a a-eye_preprocessing/ANTs/input/T1.nii.gz -e a-eye_preprocessing/ANTs/input/mni152/tpl-MNI152NLin2009cAsym_res-01_T1w.nii.gz -f a-eye_preprocessing/ANTs/input/mni152/tpl-MNI152NLin2009cAsym_res-01_desc-eye_mask.nii.gz -g a-eye_preprocessing/ANTs/input/mni152/all_segments_template_mask.nii.gz -o /mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/output/test/


registration with eye template

In [ ]:
import os, glob, subprocess

eye_img = '/mnt/sda1/Repos/a-eye/Output/eye_model/combined/output/templateConstruction/template0.nii.gz'
eye_labels = '/mnt/sda1/Repos/a-eye/Output/eye_model/combined/output/maps/nnunet/max_prob_map.nii.gz'
colin_img = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/colin27/tpl-MNIColin27_T1w.nii.gz'  # Colin27
mni_img = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/mni152/tpl-MNI152NLin2009cAsym_res-01_T1w.nii.gz'  # MNI152

for ref_img in [colin_img, mni_img]:

    ref_name = 'colin' if ref_img == colin_img else 'mni'

    output_dir = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/output/eye_atlas/' + ref_name + '/'
    os.makedirs(output_dir, exist_ok=True)

    # antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
    command1 = ['antsRegistrationSyNQuick.sh', 
                '-d', '3',
                '-m', eye_img,
                '-f', ref_img,
                '-t', 'b',
                '-o', output_dir,
                '-n', '16',
                '-j', '1']
    print(command1)
    # os.system(command1)
    process = subprocess.Popen(command1, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(f'Error: {stderr.decode()}')
    else:
        print(stdout.decode())

    # antsApplyTransforms with inverse transform to get the template labels into subject space
    command2 = ['antsApplyTransforms', 
                '-d', '3',
                '-i', eye_labels,
                '-r', ref_img,
                '-t', output_dir + '1Warp.nii.gz',
                '-t', output_dir + '0GenericAffine.mat',
                '-n', 'MultiLabel',
                '-o', output_dir + 'labels.nii.gz',
                '--float', '0',
                '--verbose', '1']
    print(command2)
    # os.system(command2)
    process = subprocess.Popen(command2, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(f'Error: {stderr.decode()}')
    else:
        print(stdout.decode())

    # Dealing with files in that folder
    # for f in glob.glob(output_image_path + 'labels.nii.gz'):
    #     os.remove(f)

crop eye region

In [ ]:
import numpy as np
import SimpleITK as sitk

colin_img = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/colin27/tpl-MNIColin27_T1w.nii.gz'  # Colin27
colin_eye_mask = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/colin27/all_segments_template_mask.nii.gz'
mni_img = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/mni152/tpl-MNI152NLin2009cAsym_res-01_T1w.nii.gz'  # MNI152
mni_eye_mask = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/mni152/all_segments_template_mask.nii.gz'

for ref_img in [colin_img, mni_img]:

    img = ref_img
    labels = colin_eye_mask if ref_img == colin_img else mni_eye_mask
    ref_name = 'colin27' if ref_img == colin_img else 'mni152'

    output_dir = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/' + ref_name + '/'

    image = sitk.ReadImage(img)
    all_segments = sitk.ReadImage(labels)
    image_x_size, image_y_size, image_z_size = image.GetSize()
    print(f"image_x_size {image_x_size} image_y_size {image_y_size} image_z_size {image_z_size}")

    bound = 15
    print(f"Bound: {bound}")

    # Mask
    all_segments_mask = all_segments > 0
    # sitk.WriteImage(all_segments_mask, base_dir+folder+'/input/'+folder+'_labels_mask.nii.gz')

    # Bounding box
    lsif = sitk.LabelStatisticsImageFilter() # It requires intensity and label images
    lsif.Execute(image, all_segments_mask) # Mask! Where all the labels are 1!
    bounding_box = np.array(lsif.GetBoundingBox(1)) # GetBoundingBox(label)
    print(f"Bounding box:  {bounding_box}") # [xmin, xmax, ymin, ymax, zmin, zmax]
    bounding_box_expanded = bounding_box.copy()
    bounding_box_expanded[0::2] -= bound # even indexes
    bounding_box_expanded[1::2] += bound # odd indexes
    print(f"Expanded bounding box: {bounding_box_expanded}")

    # Limits
    if bounding_box_expanded[0] < 0: bounding_box_expanded[0] = 0
    if bounding_box_expanded[1] > image_x_size: bounding_box_expanded[1] = image_x_size
    if bounding_box_expanded[2] < 0: bounding_box_expanded[2] = 0
    if bounding_box_expanded[3] > image_y_size: bounding_box_expanded[3] = image_y_size
    if bounding_box_expanded[4] < 0: bounding_box_expanded[4] = 0
    if bounding_box_expanded[5] > image_z_size: bounding_box_expanded[5] = image_z_size
    print(f"Expanded bounding box after limits: {bounding_box_expanded} \n")

    # Crop
    image_crop = image[int(bounding_box_expanded[0]):int(bounding_box_expanded[1]), # x
                        int(bounding_box_expanded[2]):int(bounding_box_expanded[3]), # y
                        int(bounding_box_expanded[4]):int(bounding_box_expanded[5])] # z

    sitk.WriteImage(image_crop, output_dir + 'img_cropped.nii.gz')  

registration of cropped reference images with eye atlas

In [ ]:
import os, subprocess

eye_img = '/mnt/sda1/Repos/a-eye/Output/eye_model/combined/output/templateConstruction/template0.nii.gz'
eye_labels = '/mnt/sda1/Repos/a-eye/Output/eye_model/combined/output/maps/nnunet/max_prob_map.nii.gz'
colin_img = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/colin27/img_cropped.nii.gz'  # Colin27
mni_img = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/mni152/img_cropped.nii.gz'  # MNI152

for ref_img in [colin_img, mni_img]:

    ref_name = 'colin' if ref_img == colin_img else 'mni'

    output_dir = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/output/eye_atlas/' + ref_name + '/'
    os.makedirs(output_dir, exist_ok=True)

    # antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
    command1 = ['antsRegistrationSyNQuick.sh', 
                '-d', '3',
                '-m', eye_img,
                '-f', ref_img,
                '-t', 'b',
                '-s', '10',
                '-r', '64',
                '-g', '0.15',
                '-o', output_dir,
                '-n', '16',
                '-j', '1']
    print(command1)
    # os.system(command1)
    process = subprocess.Popen(command1, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(f'Error: {stderr.decode()}')
    else:
        print(stdout.decode())

    # antsApplyTransforms with inverse transform to get the template labels into subject space
    command2 = ['antsApplyTransforms', 
                '-d', '3',
                '-i', eye_labels,
                '-r', ref_img,
                '-t', output_dir + '1Warp.nii.gz',
                '-t', output_dir + '0GenericAffine.mat',
                '-n', 'MultiLabel',
                '-o', output_dir + 'labels.nii.gz',
                '--float', '0',
                '--verbose', '1']
    print(command2)
    # os.system(command2)
    process = subprocess.Popen(command2, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(f'Error: {stderr.decode()}')
    else:
        print(stdout.decode())

    # Dealing with files in that folder
    # for f in glob.glob(output_image_path + 'labels.nii.gz'):
    #     os.remove(f)

volumes

In [ ]:
import nibabel as nb
import numpy as np
import csv

colin_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/output/eye_atlas/colin/labels.nii.gz'
mni_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/output/eye_atlas/mni/labels.nii.gz'

for labels_path in [colin_labels, mni_labels]:

    name = 'colin' if labels_path == colin_labels else 'mni'
    output_dir = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/output/eye_atlas/' + name + '/'

    # load max prob map (volume)
    labels = nb.load(labels_path)
    labels = labels.get_fdata()

    # count voxels per label
    unique, counts = np.unique(labels, return_counts=True)
    label_counts = dict(zip(unique, counts))

    # save the label_counts as a .csv file
    with open(f'{output_dir}/volumes.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['label', 'count'])
        for key, value in label_counts.items():
            writer.writerow([key, value])

inverse crop

In [ ]:
import numpy as np
import SimpleITK as sitk

colin_img = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/colin27/tpl-MNIColin27_T1w.nii.gz'  # Colin27
colin_eye_mask = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/colin27/all_segments_template_mask.nii.gz'
colin_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/output/eye_atlas/colin/labels.nii.gz'
mni_img = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/mni152/tpl-MNI152NLin2009cAsym_res-01_T1w.nii.gz'  # MNI152
mni_eye_mask = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/mni152/all_segments_template_mask.nii.gz'
mni_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/output/eye_atlas/mni/labels.nii.gz'

for ref_img in [colin_img, mni_img]:

    img = ref_img
    mask = colin_eye_mask if ref_img == colin_img else mni_eye_mask
    labels = colin_labels if ref_img == colin_img else mni_labels
    ref_name = 'colin27' if ref_img == colin_img else 'mni152'

    output_dir = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/input/' + ref_name + '/'

    image = sitk.ReadImage(img)
    all_segments = sitk.ReadImage(mask)
    image_x_size, image_y_size, image_z_size = image.GetSize()
    print(f"image_x_size {image_x_size} image_y_size {image_y_size} image_z_size {image_z_size}")

    bound = 15
    print(f"Bound: {bound}")

    # Mask
    all_segments_mask = all_segments > 0
    # sitk.WriteImage(all_segments_mask, base_dir+folder+'/input/'+folder+'_labels_mask.nii.gz')

    # Bounding box
    lsif = sitk.LabelStatisticsImageFilter() # It requires intensity and label images
    lsif.Execute(image, all_segments_mask) # Mask! Where all the labels are 1!
    bounding_box = np.array(lsif.GetBoundingBox(1)) # GetBoundingBox(label)
    print(f"Bounding box:  {bounding_box}") # [xmin, xmax, ymin, ymax, zmin, zmax]
    bounding_box_expanded = bounding_box.copy()
    bounding_box_expanded[0::2] -= bound # even indexes
    bounding_box_expanded[1::2] += bound # odd indexes
    print(f"Expanded bounding box: {bounding_box_expanded}")

    # Limits
    if bounding_box_expanded[0] < 0: bounding_box_expanded[0] = 0
    if bounding_box_expanded[1] > image_x_size: bounding_box_expanded[1] = image_x_size
    if bounding_box_expanded[2] < 0: bounding_box_expanded[2] = 0
    if bounding_box_expanded[3] > image_y_size: bounding_box_expanded[3] = image_y_size
    if bounding_box_expanded[4] < 0: bounding_box_expanded[4] = 0
    if bounding_box_expanded[5] > image_z_size: bounding_box_expanded[5] = image_z_size
    print(f"Expanded bounding box after limits: {bounding_box_expanded} \n")

    # Re-insert processed crop back into original space
    # Step 1: Load image_crop (labels)
    image_crop = sitk.ReadImage(labels)

    # Step 2: Create a blank image with the same size as the original
    blank_image = sitk.Image(image.GetSize(), image.GetPixelID())
    blank_image.CopyInformation(image)  # Copy metadata like origin, spacing, etc.
    
    # Ensure the cropped image has the same pixel type as the blank image
    image_crop = sitk.Cast(image_crop, blank_image.GetPixelID())

    # Step 3: Paste the processed crop into the blank image
    start_index = [int(bounding_box_expanded[0]), int(bounding_box_expanded[2]), int(bounding_box_expanded[4])]
    blank_image[start_index[0]:start_index[0] + image_crop.GetSize()[0],
                start_index[1]:start_index[1] + image_crop.GetSize()[1],
                start_index[2]:start_index[2] + image_crop.GetSize()[2]] = image_crop

    # Step 4: Save the image with the crop placed back in the original space
    sitk.WriteImage(blank_image, output_dir + 'seg_atlas.nii.gz')